In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import warnings

# 경고 무시 및 시드 설정
warnings.filterwarnings('ignore')
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# MPS 장치 설정 (Apple Silicon 가속)
if torch.backends.mps.is_available():
    device = 'mps'
    print(f"✅ MPS 가속이 활성화되었습니다.")
else:
    device = 'cpu'
    print(f"⚠️ MPS를 찾을 수 없어 CPU로 실행합니다.")

# ==========================================
# 1. 데이터 로드
# ==========================================
train_df = pd.read_csv("../Data/train.csv")
test_df = pd.read_csv("../Data/test.csv")

# ==========================================
# 2. 전처리 함수 (User Provided Logic + Bug Fix)
# ==========================================
def preprocess_for_tabnet(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        if "IUI" in x: return "IUI"
        if "DI" in x: return "Other"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부 관련
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 배아 진행 단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0: return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0: return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0: return '이식_미실시'
        else: return '이식_완료'
    
    df_copy['배아_진행_단계'] = df_copy.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회': return '0회'
        elif x in ['1회', '2회']: return '1–2회'
        else: return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음': return 'Unknown'
        elif age == '만18-34세': return '34세 이하'
        elif age in ['만35-37세', '만38-39세']: return '35-39세'
        else: return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return '0개'
        elif count <= 2: return '1-2개'
        else: return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    # 배아 효율 비율 변수들
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # 교호작용 변수들
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + df_copy['Day5_이식_여부'].astype(str)
    df_copy['시술횟수×나이'] = df_copy['총시술_bin3'] + '_' + df_copy['나이_3구간']

    # ID 컬럼 제거 (제공된 코드의 버그 수정: df_copy에서 제거)
    if 'ID' in df_copy.columns:
        df_copy = df_copy.drop(['ID'], axis=1)
    
    # 배아_이식_미도달은 파생변수 생성용이므로 제거
    if '배아_이식_미도달' in df_copy.columns:
        df_copy = df_copy.drop(['배아_이식_미도달'], axis=1)
        
    return df_copy

# 전처리 적용
train_processed = preprocess_for_tabnet(train_df)
test_processed = preprocess_for_tabnet(test_df)

# 타겟 분리
target_col = '임신 성공 여부'
X = train_processed.drop(columns=[target_col])
y = train_processed[target_col].values
X_test = test_processed.copy()

# ==========================================
# 3. 데이터 누수 방지 (Imputation, Encoding, Scaling)
# ==========================================

# 컬럼 타입 자동 분류
numeric_cols = X.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 3-1. 결측치 처리 (Numeric) - Train의 Median 활용
# [누수 방지 준수 6번]: Test 결측치는 Train 통계값으로 처리
imputer_num = SimpleImputer(strategy='median')
X[numeric_cols] = imputer_num.fit_transform(X[numeric_cols])
X_test[numeric_cols] = imputer_num.transform(X_test[numeric_cols])

# 3-2. 결측치 처리 (Categorical) - 'Unknown'으로 채움
X[categorical_cols] = X[categorical_cols].fillna("Unknown")
X_test[categorical_cols] = X_test[categorical_cols].fillna("Unknown")

# 3-3. Label Encoding (TabNet 필수)
# [누수 방지 준수 3번, 5번]: Test 데이터는 fit에 사용하지 않음
categorical_dims = {}
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(X[col].astype(str))
    
    # Train 변환
    X[col] = le.transform(X[col].astype(str))
    
    # Test 변환 (Unknown Label Handling)
    # Test에만 있는 새로운 카테고리는 Train의 최빈값(mode) 또는 0번 클래스로 매핑
    test_values = X_test[col].astype(str).values
    # 학습된 클래스에 없는 값 식별
    unseen_mask = ~np.in1d(test_values, le.classes_)
    
    if unseen_mask.any():
        # Train의 최빈값 찾기
        mode_val = X[col].mode()[0] 
        # unseen 값을 임시로 classes_[0]로 대체 (transform 에러 방지용) 후 나중에 최빈값으로 덮어씀
        # 여기서는 단순히 transform 가능한 값으로 매핑하고 인코딩 후 mode_val(integer)로 교체
        
        # 1. 안전하게 변환하기 위해 unseen을 known class 중 하나로 임시 변경
        temp_safe_val = le.classes_[0]
        test_values_safe = test_values.copy()
        test_values_safe[unseen_mask] = temp_safe_val
        
        # 2. 변환 수행
        X_test[col] = le.transform(test_values_safe)
        
        # 3. Unseen 위치에 Train Mode(이미 인코딩된 정수) 할당
        X_test.loc[unseen_mask, col] = mode_val
    else:
        X_test[col] = le.transform(test_values)
        
    categorical_dims[col] = len(le.classes_)

# 3-4. Scaling (Numeric)
# [누수 방지 준수 4번]: Scaler는 Train에만 Fit
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# 메모리 효율을 위해 float32로 변환
X = X.astype('float32')
X_test = X_test.astype('float32')

# ==========================================
# 4. TabNet 준비 (인덱스 및 파라미터)
# ==========================================

# Categorical Feature Index 추출 (TabNet 입력용)
cat_idxs = [i for i, f in enumerate(X.columns) if f in categorical_cols]
cat_dims = [categorical_dims[f] for f in categorical_cols]

# 학습/검증 데이터 분리 (Stratified Split)
# [누수 방지 준수 1번]: 모델 학습 시 검증용 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(
    X.values, y, test_size=0.2, random_state=SEED, stratify=y
)

# 불균형 데이터 가중치 계산 (Positive 클래스 가중치)
# 0: 190123, 1: 66228 => 비율 약 2.87배
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
class_weights = torch.tensor([1.0, pos_weight], dtype=torch.float).to(device)
print(f"Computed Positive Class Weight: {pos_weight:.4f}")

# ==========================================
# 5. TabNet 모델 학습
# ==========================================

clf = TabNetClassifier(
    n_d=32, n_a=32, n_steps=5,    # 모델 복잡도 조절
    gamma=1.5, n_independent=2, n_shared=2,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=1,                # 범주형 임베딩 차원
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',           # entmax or sparsemax
    device_name=device,           # MPS 설정
    seed=SEED
)

# Weighted Loss Function 정의
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=100,                # 필요에 따라 조절 (예: 100)
    patience=10,
    batch_size=1024,              # 대용량 데이터 배치 사이즈
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    loss_fn=loss_fn               # 가중치 적용된 손실함수 전달
)

# ==========================================
# 6. Test Data 예측
# ==========================================

preds_proba = clf.predict_proba(X_test.values)
final_probs_tabnet = preds_proba[:, 1] # Positive 클래스(1)에 대한 확률만 추출


✅ MPS 가속이 활성화되었습니다.
Computed Positive Class Weight: 2.8707
epoch 0  | loss: 0.65653 | train_auc: 0.71722 | valid_auc: 0.71646 |  0:00:44s
epoch 1  | loss: 0.59411 | train_auc: 0.71849 | valid_auc: 0.71622 |  0:01:27s
epoch 2  | loss: 0.58958 | train_auc: 0.72514 | valid_auc: 0.72232 |  0:02:10s
epoch 3  | loss: 0.58876 | train_auc: 0.72266 | valid_auc: 0.71873 |  0:02:53s
epoch 4  | loss: 0.58887 | train_auc: 0.72603 | valid_auc: 0.72377 |  0:03:34s
epoch 5  | loss: 0.58581 | train_auc: 0.72945 | valid_auc: 0.72555 |  0:04:17s
epoch 6  | loss: 0.58519 | train_auc: 0.72866 | valid_auc: 0.7253  |  0:05:01s
epoch 7  | loss: 0.58355 | train_auc: 0.73194 | valid_auc: 0.72803 |  0:05:42s
epoch 8  | loss: 0.58179 | train_auc: 0.73199 | valid_auc: 0.72828 |  0:06:25s
epoch 9  | loss: 0.58157 | train_auc: 0.73366 | valid_auc: 0.7297  |  0:07:08s
epoch 10 | loss: 0.58074 | train_auc: 0.73438 | valid_auc: 0.72963 |  0:07:51s
epoch 11 | loss: 0.57995 | train_auc: 0.73472 | valid_auc: 0.72995 |  0:

In [3]:
# ==========================================
# 7. 결과 추출
# ==========================================
from datetime import datetime
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"TN_{now}_submission.csv"

submission = pd.read_csv("../Data/sample_submission.csv")
submission['probability'] = final_probs_tabnet
submission.to_csv(file_name, index=False)

print(f"완료! 결과 저장됨: {file_name}")
print(f"Best Validation AUC: {clf.best_cost}")

완료! 결과 저장됨: TN_0210_2321_submission.csv
Best Validation AUC: 0.7367179542770432
